# 📚 Comprehensive Fine-Tuning Pipeline: BART-LoRA for HighlightSum

## Overview
This notebook implements a complete **end-to-end fine-tuning pipeline** for abstractive summarization using **BART-Large** with **LoRA adapters**. It includes:
- ✅ Dataset loading & inspection
- ✅ Multi-model benchmarking
- ✅ Fine-tuning with PEFT
- ✅ Comprehensive evaluation (ROUGE, BERTScore, BLEU)
- ✅ Error analysis & visualization
- ✅ Model merging & inference

**Target Task**: Highlight-based abstractive summarization on HighlightSum dataset
**Hardware**: GPU (T4, L4, or better recommended)


# Dataset Inspection: HighlightSum

This section loads and explores the **HighlightSum** dataset to understand its structure and contents.

In [1]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("knkarthick/highlightsum")

# View first sample structure
sample = dataset["train"][0]
print("📊 Dataset Overview:")
print(f"  Train splits: {len(dataset['train']):,} samples")
print(f"  Val splits: {len(dataset['validation']):,} samples")
print(f"  Test splits: {len(dataset['test']):,} samples")
print(f"\n🔑 Keys: {list(sample.keys())}")

# Inspect sample details
print("\n📘 First training example:")
print(f"\n🔸 DIALOGUE ({len(sample['dialogue'])} chars):\n{sample['dialogue'][:300]}...")
print(f"\n🔹 SUMMARY ({len(sample['summary'])} chars):\n{sample['summary']}")


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/27.0M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/27401 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1360 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2347 [00:00<?, ? examples/s]

📊 Dataset Overview:
  Train splits: 27,401 samples
  Val splits: 1,360 samples
  Test splits: 2,347 samples

🔑 Keys: ['id', 'dialogue', 'summary']

📘 First training example:

🔸 DIALOGUE (32390 chars):
Speaker A: Cool. Do you wanna give me the little cable thing? Yeah. Cool. Ah, that's why it won't meet. Okay, cool. Yep, cool. Okay, functional requirements. Alright, yeah. It's working. Cool, okay. So what I have, wh where I've got my information from is a survey where the usability lab um observed...

🔹 SUMMARY (1299 chars):
The project manager opens the meeting by stating that they will address functional design and then going over the agenda. The industrial designer gives his presentation, explaining how remote controls function and giving personal preference to a clear, simple design that upgrades the technology as well as incorporates the latest features in chip design. The interface specialist gives her presentation next, addressing the main purpose of a remote control. She pinpoin

# Large Model Safety Benchmarking (with Preprocessing)

This section outlines the preprocessing steps required before running benchmarking or fine‑tuning large models. Since only **2,000 samples** will be used for fine‑tuning, efficient batching and GPU-aware preprocessing are essential.

### Why Preprocessing Matters

Preprocessing ensures that:
- Inputs are normalized and consistently formatted.
- Tokenization is performed efficiently before training.
- Dataset splits are clearly defined.

This improves both reproducibility and training performance.

_Basic preprocessing_ has been performed, including tokenization of dialogues with appropriate padding and truncation, batch preparation for seq2seq models, and selection of a subset from the HighlightSUM train split for benchmarking. In details, the performed basic preprocessing is charachterised by:  
> _Tokenization_:
  - All text inputs (dialogue) are tokenized using the model-specific tokenizer.  
  - For causal models, if pad_token was missing, it was set to eos_token to allow batching/padding.  
  - Seq2seq and causal models both use truncation and padding (max_length=1024) to ensure consistent tensor shapes.  
> Dataset splitting:  
  - Selected a subset of samples (N_SAMPLES) from the HighlightSUM train split for benchmarking.
  - If desired, you could extend this to full train/validation/test splits for proper evaluation.
> Batching (seq2seq models):  
  - Inputs are batched to reduce memory usage on GPU, which is part of preprocessing before model inference.

In [4]:
# =====================================================
# ᐡ Notebook C — Large Model Safe Benchmarking (Refactored)
# =====================================================
# Usage: paste into Google Colab and run.
# Notes:
#  - Designed to be Colab-friendly (offload support, small sample defaults).
#  - Batches causal models for better throughput.
#  - Seeds for reproducibility.
#  - Standardized metric normalization (z-score) for composite scoring.
# =====================================================

# -------------------------
# Install dependencies (run once)
# -------------------------
!pip install -q datasets transformers accelerate sentencepiece rouge-score pandas tqdm matplotlib seaborn

# -------------------------
# Imports
# -------------------------
import os
import time
import json
import random
import math
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from rouge_score import rouge_scorer

# -------------------------
# Configuration (tweak here)
# -------------------------
DEBUG = False                 # set True to enable CUDA_LAUNCH_BLOCKING for debugging
USE_OFFLOAD = True            # set True to allow model offloading (helps on Colab)
OFFLOAD_FOLDER = "offload"    # folder for offloaded tensors
N_SAMPLES = 200               # number of test samples to evaluate (use small number for quick runs)
SEQ2SEQ_BATCH_SIZE = 4        # batch size for seq2seq (e.g., BART, T5)
CAUSAL_BATCH_SIZE = 2         # batch size for causal (e.g., LLaMA-like) -- keep small for memory
#MAX_INPUT_TOKENS = 1024       # tokenizer max length for inputs
MAX_INPUT_TOKENS = 768
#MAX_NEW_TOKENS = 120          # tokens to generate
MAX_NEW_TOKENS = 192
#SAVE_DIR = "outputs/benchmarks/notebook_C"
SAVE_DIR = "/content/llmed_certification_FineTuneFlow/outputs/benchmarks/notebook_C"
os.makedirs(SAVE_DIR, exist_ok=True)

# -------------------------
# Optional debug env
# -------------------------
if DEBUG:
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# -------------------------
# Device
# -------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"ኂ Using device: {device}")

# -------------------------
# Reproducibility
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -------------------------
# Load dataset (small subset by default)
# -------------------------
dataset = load_dataset("knkarthick/highlightsum")
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"].select(range(min(N_SAMPLES, len(dataset["test"]))))
print("Train samples:", len(train_data))
print("Validation samples:", len(val_data))
print("Test samples (used):", len(test_data))
print("Columns:", test_data.column_names)

# -------------------------
# ROUGE scorer
# -------------------------
scorer = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)

def compute_rouge(preds, refs):
    """
    Compute average ROUGE f-measures (percent).
    Uses rouge_scorer where signature is: score(target/reference, prediction)
    """
    assert len(preds) == len(refs), "Predictions and references must match length"
    agg = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}
    for pred, ref in zip(preds, refs):
        # target = reference, prediction = model output
        scores = scorer.score(ref, pred)
        for k in agg:
            agg[k] += scores[k].fmeasure
    n = len(preds) if len(preds) > 0 else 1
    return {k: (v / n) * 100.0 for k, v in agg.items()}

# -------------------------
# Utilities
# -------------------------
def batchify(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i+batch_size]

def safe_tokenizer_pad(tokenizer):
    if tokenizer.pad_token is None:
        # ensure a pad token exists for batching
        if tokenizer.eos_token is not None:
            tokenizer.pad_token = tokenizer.eos_token
        else:
            tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    return tokenizer

# -------------------------
# Summarization inference functions
# -------------------------
def summarize_seq2seq_batch(model, tokenizer, texts, max_input_tokens=MAX_INPUT_TOKENS, max_new_tokens=MAX_NEW_TOKENS):
    """
    Perform batched seq2seq inference. Returns (pred_texts, elapsed_time_seconds).
    Measures only the generation time (tokenization done outside timer).
    """
    # Tokenize (on CPU/GPU depending on tokenizer implementation)
    inputs = tokenizer(
        texts,
        truncation=True,
        padding="longest",
        max_length=max_input_tokens,
        return_tensors="pt"
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    start = time.time()
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    latency = time.time() - start
    preds = [tokenizer.decode(o, skip_special_tokens=True).strip() for o in output_ids]
    return preds, latency

def summarize_causal_batch(model, tokenizer, texts, system_prompt="Summarize this text concisely.", max_input_tokens=MAX_INPUT_TOKENS, max_new_tokens=MAX_NEW_TOKENS):
    """
    Perform batched inference for causal/chat-style models. Accepts a list of raw texts.
    We create a simple prompt; for more advanced chat tokenizers, a tokenizer-specific template may be needed.
    Returns (pred_texts, elapsed_time_seconds).
    """
    # Build prompts: simple system + user text; if tokenizer has chat helpers you can extend this.
    prompts = []
    for t in texts:
        # Use a modest prompt that works with most causal models
        p = f"{system_prompt}\n\nText:\n{t}\n\nSummary:"
        prompts.append(p)

    inputs = tokenizer(
        prompts,
        truncation=True,
        padding="longest",
        max_length=max_input_tokens,
        return_tensors="pt"
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    torch.cuda.synchronize() if torch.cuda.is_available() else None
    start = time.time()
    out_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.pad_token_id)
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    latency = time.time() - start
    preds = [tokenizer.decode(o, skip_special_tokens=True).strip() for o in out_ids]
    # Post-process: remove the prompt prefix if tokenizer doesn't handle it (best-effort)
    # We'll do a simple split by "Summary:" and take the tail if present
    cleaned = []
    for prompt, pred in zip(prompts, preds):
        if "Summary:" in pred:
            cleaned.append(pred.split("Summary:",1)[-1].strip())
        else:
            # fallback: try to remove repeating prompt portion
            cleaned.append(pred.replace(prompt, "").strip()[:1000])
    return cleaned, latency

# -------------------------
# Model benchmark core
# -------------------------
def load_model_safe(model_id, use_offload=USE_OFFLOAD, torch_dtype=torch.float16):
    """
    Load a model with device_map='auto'. If offload enabled, set offload_folder.
    Returns the loaded model and a flag is_seq2seq.
    """
    # attempt to detect seq2seq by config when possible
    # We try Seq2Seq loader first only if model config indicates encoder-decoder
    kwargs = {"device_map": "auto", "torch_dtype": torch_dtype}
    if use_offload:
        os.makedirs(OFFLOAD_FOLDER, exist_ok=True)
        kwargs["offload_folder"] = OFFLOAD_FOLDER

    # We'll try to load the model with generic loaders. Errors are caught by caller.
    # Load as seq2seq if model config indicates encoder-decoder (preferred detection).
    model_try = None
    # We need to inspect remote config -> using AutoModelForSeq2SeqLM may fail on causal models,
    # so we try seq2seq first and fallback to causal.
    try:
        model_try = AutoModelForSeq2SeqLM.from_pretrained(model_id, **kwargs)
        is_seq2seq = getattr(model_try.config, "is_encoder_decoder", True)
        return model_try, is_seq2seq
    except Exception:
        # fallback to causal
        model_try = AutoModelForCausalLM.from_pretrained(model_id, **kwargs)
        is_seq2seq = getattr(model_try.config, "is_encoder_decoder", False)
        return model_try, is_seq2seq

def benchmark_models_safe(models_dict, n_samples=N_SAMPLES, seq2seq_bs=SEQ2SEQ_BATCH_SIZE, causal_bs=CAUSAL_BATCH_SIZE):
    """
    Benchmark multiple models safely.
    Returns a dict of results per model.
    Each model result contains: rouge1, rouge2, rougeL, time, throughput, efficiency
    """
    results = {}
    refs = test_data["summary"][:n_samples]

    for name, model_id in models_dict.items():
        print(f"\nὨ0 Benchmarking {name} ({model_id}) ...")
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
            tokenizer = safe_tokenizer_pad(tokenizer)

            # Load model
            model, is_seq2seq = load_model_safe(model_id)
            model.eval()
            # ensure model uses tokenizer length if we added tokens
            try:
                model.resize_token_embeddings(len(tokenizer))
            except Exception:
                pass

            # Move model to correct device is handled by device_map="auto" loader.
            # But in CPU-only environment we move to CPU device explicitly.
            if not torch.cuda.is_available():
                model.to(device)

            preds = []
            total_time = 0.0

            if is_seq2seq:
                # Batch processing for seq2seq models
                indices = list(range(0, n_samples))
                for i in tqdm(range(0, n_samples, seq2seq_bs), desc=f"{name} (seq2seq)"):
                    batch_texts = test_data["dialogue"][i:i+seq2seq_bs]
                    batch_preds, latency = summarize_seq2seq_batch(model, tokenizer, batch_texts)
                    preds.extend(batch_preds)
                    total_time += latency
            else:
                # Batch processing for causal/chat models
                for i in tqdm(range(0, n_samples, causal_bs), desc=f"{name} (causal)"):
                    batch_texts = test_data["dialogue"][i:i+causal_bs]
                    batch_preds, latency = summarize_causal_batch(model, tokenizer, batch_texts)
                    preds.extend(batch_preds)
                    total_time += latency

            # ensure correct length
            preds = preds[:n_samples]
            refs_used = refs[:len(preds)]

            # Compute metrics
            scores = compute_rouge(preds, refs_used)
            total_time = max(total_time, 1e-8)  # avoid div by zero
            throughput = len(preds) / total_time
            efficiency = scores["rougeL"] / total_time

            results[name] = {
                "model_id": model_id,
                "rouge1": scores["rouge1"],
                "rouge2": scores["rouge2"],
                "rougeL": scores["rougeL"],
                "time": total_time,
                "throughput": throughput,
                "efficiency": efficiency
            }

        except Exception as e:
            print(f"❌ Error with {name}: {e}")
            results[name] = {
                "model_id": model_id,
                "rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0,
                "time": float("inf"), "throughput": 0.0, "efficiency": 0.0
            }

        # Free GPU memory after each model if available
        try:
            del model
            torch.cuda.empty_cache()
        except Exception:
            pass

    return results

# -------------------------
# Models to benchmark (example sets)
# -------------------------
MODELS_A = {
    "BART-large": "facebook/bart-large-cnn",
    "T5-large": "t5-large",
}

MODELS_B = {
    # Replace with models accessible in your environment; some may not be downloadable in limited Colab.
    "Phi-3-Mini": "microsoft/Phi-3-mini-4k-instruct",
    "LLaMA-1B": "meta-llama/Llama-3.2-1B-Instruct",
    "LLaMA-3B": "meta-llama/Llama-3.2-3B-Instruct",
}

# =====================================================
# RUN BENCHMARK (A then B)
# =====================================================
print("\nኂ Running Model Benchmarks...")
results_A = benchmark_models_safe(MODELS_A, n_samples=min(N_SAMPLES, len(test_data)))
results_B = benchmark_models_safe(MODELS_B, n_samples=min(N_SAMPLES, len(test_data)))
results_all = {**results_A, **results_B}

# =====================================================
# Build DataFrame and compute composite score
# =====================================================
df = pd.DataFrame.from_dict(results_all, orient="index")
# Ensure numeric types
for col in ["rouge1","rouge2","rougeL","time","throughput","efficiency"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0)

# Standardize (z-score) selected metrics to make composite score robust to scale
metrics_for_norm = ["rougeL", "efficiency", "throughput"]
df_norm = df.copy()
for col in metrics_for_norm:
    if df[col].std() > 0:
        df_norm[col] = (df[col] - df[col].mean()) / df[col].std()
    else:
        df_norm[col] = 0.0

# Composite: weights (tunable)
df["composite_score"] = (
    0.60 * df_norm["rougeL"]
    + 0.25 * df_norm["efficiency"]
    + 0.15 * df_norm["throughput"]
)

df_sorted = df.sort_values("composite_score", ascending=False)
display(df_sorted)

# =====================================================
# Save results
# =====================================================
os.makedirs(SAVE_DIR, exist_ok=True)
csv_path = os.path.join(SAVE_DIR, "final_ranking.csv")
html_path = os.path.join(SAVE_DIR, "final_ranking.html")
json_path = os.path.join(SAVE_DIR, "final_ranking.json")

df_sorted.to_csv(csv_path)
df_sorted.to_html(html_path)
with open(json_path, "w") as f:
    json.dump(df_sorted.to_dict(orient="index"), f, indent=2)

print(f"\nSaved outputs to {SAVE_DIR}")


  Preparing metadata (setup.py) ... done
ኂ Using device: cuda
Train samples: 27401
Validation samples: 1360
Test samples (used): 200
Columns: ['id', 'dialogue', 'summary']

ኂ Running Model Benchmarks...

Ὠ0 Benchmarking BART-large (facebook/bart-large-cnn) ...


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LLaMA-3B (causal): 100%|██████████| 100/100 [12:29<00:00,  7.50s/it]


,model_id,rouge1,rouge2,rougeL,time,throughput,efficiency,composite_score
BART-large,facebook/bart-large-cnn,28.105383,9.183429,21.062636,101.631836,1.967887,0.207244,1.230694
LLaMA-1B,meta-llama/Llama-3.2-1B-Instruct,28.635874,9.618125,21.205387,393.929495,0.507705,0.053830,0.463230
LLaMA-3B,meta-llama/Llama-3.2-3B-Instruct,23.771793,8.222793,17.306203,748.223488,0.267300,0.023130,-0.162342
Phi-3-Mini,microsoft/Phi-3-mini-4k-instruct,20.550442,7.028457,14.306677,987.636199,0.202504,0.014486,-0.571619
T5-large,t5-large,10.977282,1.944009,9.636944,263.027842,0.760376,0.036638,-0.959962



Saved outputs to /content/llmed_certification_FineTuneFlow/outputs/benchmarks/notebook_C
